In [ ]:
%pip install snowflake-connector-python 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pyopenssl to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 90.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall:

In [ ]:
import os
import pandas as pd
import snowflake.connector

conn = snowflake.connector.connect(
    user=os.environ["SNOWFLAKE_USER"],
    password=os.environ["SNOWFLAKE_PASSWORD"],
    account=os.environ["SNOWFLAKE_ACCOUNT"],
    warehouse=os.environ["SNOWFLAKE_WAREHOUSE"],
    database=os.environ["SNOWFLAKE_DATABASE"],
    schema="PUBLIC"
)


In [ ]:
query = 'SELECT * FROM "GENDER_DATA"' #table_name
df=pd.read_sql(query , conn)

conn.close()

df.head()

/tmp/ipython-input-2317232195.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query , conn)


,LONG_HAIR,FOREHEAD_WIDTH_CM,FOREHEAD_HEIGHT_CM,NOSE_WIDE,NOSE_LONG,LIPS_THIN,DISTANCE_NOSE_TO_LIP_LONG,GENDER
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female
2,0,11.8,6.3,1,1,1,1,Male
3,0,14.4,6.1,0,1,1,1,Male
4,1,13.5,5.9,0,0,0,0,Female


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [6]:
df.isna().sum()

,0
LONG_HAIR,0
FOREHEAD_WIDTH_CM,0
FOREHEAD_HEIGHT_CM,0
NOSE_WIDE,0
NOSE_LONG,0
LIPS_THIN,0
DISTANCE_NOSE_TO_LIP_LONG,0
GENDER,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   LONG_HAIR                  5001 non-null   int64  
 1   FOREHEAD_WIDTH_CM          5001 non-null   float64
 2   FOREHEAD_HEIGHT_CM         5001 non-null   float64
 3   NOSE_WIDE                  5001 non-null   int64  
 4   NOSE_LONG                  5001 non-null   int64  
 5   LIPS_THIN                  5001 non-null   int64  
 6   DISTANCE_NOSE_TO_LIP_LONG  5001 non-null   int64  
 7   GENDER                     5001 non-null   object 
dtypes: float64(2), int64(5), object(1)
memory usage: 312.7+ KB


In [8]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
df['GENDER']=le.fit_transform(df['GENDER'])

In [9]:
df['GENDER'].dtypes

dtype('int64')

In [10]:
x=df.drop(['GENDER'],axis=1)
y=df['GENDER']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [11]:
m1 = LogisticRegression()
m1.fit(x_train, y_train)
y_pred = m1.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9600399600399601
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.97      0.96       502
           1       0.97      0.95      0.96       499

    accuracy                           0.96      1001
   macro avg       0.96      0.96      0.96      1001
weighted avg       0.96      0.96      0.96      1001

Confusion Matrix:
 [[485  17]
 [ 23 476]]


In [13]:
import pickle

with open('logistic_regression_model_gender_class.pkl', 'wb') as f:
    pickle.dump(m1, f)

In [14]:
with open('logistic_regression_model_gender_class.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [15]:
import joblib

joblib.dump(m1, 'logistic_regression_model_joblib.pkl')

loaded_model_joblib = joblib.load('logistic_regression_model_joblib.pkl')